In [1]:
import dill
import math
from collections import defaultdict
import csv
import itertools
import random
import multiprocessing
import csv
import utils

In [2]:
%%time

category_products = defaultdict(lambda: [])
with open('/data/python-scrapy-data-mining/corpus.csv') as raw_file:
    for (prod_id, _, url) in csv.reader(raw_file):
        url = url.split('/')
        if url[3] != 'p':
            continue
        category_products['/'.join(url[4:-1])].append(int(prod_id))
        #product_category[int(prod_id)] = '/'.join(url[4:-1])

CPU times: user 868 ms, sys: 12 ms, total: 880 ms
Wall time: 1.28 s


In [3]:
print "Total products:", sum( (len(prods) for cat, prods in category_products.iteritems()) )
print "Total categories:", len(category_products)

Total products: 228562
Total categories: 656


In [ ]:
#category_products = sorted(((c, p) for (p, c) in product_category.items()), key=lambda e: e[0])
#category_products = [(c, [p for (_, p) in pi]) for c, pi in itertools.groupby(category_product, key=lambda p: p[0])]

In [4]:
cat_counts = ((c, len(prods)) for (c, prods) in sorted(category_products.items(), key=lambda e: -len(e[1])))
cat_counts = sorted(cat_counts, key=lambda e: e[1])

In [5]:
cat_counts[:10] + cat_counts[-10:]

[('sepeda/wheel-hub-rims-dll/bearing', 1),
 ('perlengkapan-bayi/makanan-711/jus', 1),
 ('onderdil-motor/sparepart-motor/dinamo', 1),
 ('onderdil-mobil', 1),
 ('hobi/olahraga', 1),
 ('sepeda/steering/handlepost', 1),
 ('sepeda/outwear', 1),
 ('onderdil-mobil/interior-mobil', 1),
 ('sepeda/brake/cantilever', 1),
 ('onderdil-mobil/produk-perawatan-mobil', 1),
 ('hobi/mainan/diecast', 3395),
 ('fashion/pria/sepatu-169', 3494),
 ('rumah-tangga/home-stuff', 3976),
 ('fashion/wanita/tas-wanita', 4211),
 ('personal-care/produk-kesehatan', 4672),
 ('rumah-tangga/dapur', 4803),
 ('elektronik/lain-lain-208', 5011),
 ('fashion/pria/jam-tangan-171', 6208),
 ('handphone/aksesoris-handphone', 6827),
 ('handphone/case-cover', 9072)]

In [6]:
%%time
products = utils.restore_object("products.pkl")

CPU times: user 7.09 s, sys: 224 ms, total: 7.32 s
Wall time: 7.34 s


In [7]:
# category list with at least 2 products:
categories = utils.get_categories()

In [8]:
def save_sims(category, sims):
    with open("/data/python-scrapy-data-mining/sims_{0}.csv".format(category.replace('/','_')), "w") as f:
        csv_writer = csv.writer(f)
        csv_writer.writerows(sims)

def get_sim(categories):
    global category_products, products
    for cat in categories:
        sim = []
        prod_ids = sorted(category_products[cat])
        # ignore categories with a single product:
        if len(prod_ids) < 2:
            continue
        for i in prod_ids[:-1]:
            for j in (p for p in prod_ids if p > i):
                try: 
                    dist = utils.distance(products[i], products[j])
                    if dist > 0.3:
                        sim.append((i, j, round(dist, 7)))
                except Exception as e:
                    print e
                    print "***", i, j
        # Store in a CSV file and free the memory:
        if sim:
            save_sims(cat, sim)
        #print "*** Category:", cat, "of", len(prod_ids), "products with ", len(sim), "similarities"
        del(sim)

In [10]:
%%time
get_sim(categories[:10])

CPU times: user 2.36 s, sys: 8 ms, total: 2.37 s
Wall time: 2.38 s


In [ ]:
#Test get_sim:
#get_sim([cat for cat, count in cat_counts if count > 10][:5])

In [11]:
%%time

random.shuffle(categories)
pool = multiprocessing.Pool(processes=4)
pool.map(get_sim, utils.chunks(categories, 4))

CPU times: user 328 ms, sys: 164 ms, total: 492 ms
Wall time: 6min 29s
